# Product Recommender on RetailRocket Dataset

- We have used the cornac package and mircosoft's recommenders module to train a Bayesian Personalised Ranking model on retail rocket e commere dataset. 

- The model learns and recommends top K items after ranking them based on user and product interactions

- Dataset in itself is huge, hence we have taken a subsample to train the model on google colab

In [1]:
AUTHORNAME = "Archit Kaila"
COLLABORATORS = "Shrey Gupta, Shen Juin Lee"

In [2]:
## Clone the repository and code base to run Non DRL Recommenders
!git clone --single-branch --branch test https://github.com/architkaila/recommenders_aipi590.git

Cloning into 'recommenders_aipi590'...
remote: Enumerating objects: 233, done.
remote: Counting objects: 100% (233/233), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 233 (delta 116), reused 183 (delta 72), pack-reused 0
Receiving objects: 100% (233/233), 96.45 KiB | 8.04 MiB/s, done.
Resolving deltas: 100% (116/116), done.


In [3]:
## Install required libraries (only for google colab)
!pip install git+https://github.com/textomatic/cornac.git
!pip install recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/textomatic/cornac.git to /tmp/pip-req-build-lx3bf_o2
  Running command git clone -q https://github.com/textomatic/cornac.git /tmp/pip-req-build-lx3bf_o2
  Created wheel for cornac: filename=cornac-1.14.2-cp38-cp38-linux_x86_64.whl size=14329810 sha256=d16c96452897cdb8a04b1cc8c5745263426722c507ba94de132e5c1a9a11442f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ege64e_f/wheels/ce/60/14/a887f00b396951c22e4e119ac64935f8619aa113312bb949b5
Successfully built cornac
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 339 kB 34.9 MB/s 
     |████████████████████████████████| 771 kB 56.2 MB/s 
     |████████████████████████████████| 355 kB 70.3 MB/s 
     |████████████████████████████████| 662 kB 68.6 MB/s 
     |████████████████████████████████| 61 kB 9.0 MB/s 
     |██████████

In [4]:
## Fetch the dataset from S3 Bucket
!wget https://aipi590.s3.amazonaws.com/events.csv -P "/content/recommenders_aipi590/Non_DRL_Recommenders/Dataset_1_Retail_Rocket/"


--2022-12-15 15:59:26--  https://aipi590.s3.amazonaws.com/events.csv
Resolving aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)... 3.5.17.214, 52.216.27.108, 52.217.83.156, ...
Connecting to aipi590.s3.amazonaws.com (aipi590.s3.amazonaws.com)|3.5.17.214|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94237913 (90M) [text/csv]
Saving to: ‘/content/recommenders_aipi590/Non_DRL_Recommenders/Dataset_1_Retail_Rocket/events.csv’

events.csv          100%[===================>]  89.87M  12.3MB/s    in 8.9s    

2022-12-15 15:59:36 (10.1 MB/s) - ‘/content/recommenders_aipi590/Non_DRL_Recommenders/Dataset_1_Retail_Rocket/events.csv’ saved [94237913/94237913]



In [5]:
## Import standard libraries
import pandas as pd
import numpy as np

In [6]:
## Import python script to run and evaluate BPR model
from recommenders_aipi590.Non_DRL_Recommenders.bpr_model import run_bpr_model

### Read dataset

In [7]:
## Reading the e-commerce dataset
df = pd.read_csv('/content/recommenders_aipi590/Non_DRL_Recommenders/Dataset_1_Retail_Rocket/events.csv')
df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [8]:
## The implicit feedback between items and users pairs can be obtained using the events column
df.event.value_counts()

view           2664312
addtocart        69332
transaction      22457
Name: event, dtype: int64

In [9]:
## We take a subsample of our original dataset to train out model
df = df.sample(n=5000, random_state=0)

### Prepare datset

- The BPR implimentation in Cornac module works on the rankings (implicit feedbacks) for each user item pair. 

- We use the Negative Sampling method to prepare our data. This works on the assumption that if there is a interaction between user and item, then ranking is set to one else it is set to 0

- The postive interactions are present in our dataset and the negative interactions we prepare manually

In [10]:
## Set ranking (implicit feedbak) to 1 for interactions between user and item
df = df[['visitorid', 'itemid']].copy()
df['FEEDBACK'] = 1

# Remove duplicates from our samples
df = df.drop_duplicates()

# Rename the columns for explanability
df.rename(columns = {'visitorid': 'userID', 'itemid': 'itemID', 'FEEDBACK': 'rating'}, inplace = True)

df.head()

,userID,itemID,rating
125223,1034162,392595,1
2744309,203418,409876,1
1829628,1023669,325670,1
43129,1290714,229204,1
1261516,168692,220360,1


In [11]:
## Obtain list of unique items and users present in our dataset to genrate negative interations
item_ids = df['itemID'].unique()
user_ids = df['userID'].unique()

In [12]:
## Adding negative feedback (0 ranking) for instances of no interaction between items and users
absent_interactions_feedback = [[user, item, 0] for item in item_ids for user in user_ids] 

# Convert prepared data into a dataframe
negative_feedback_df = pd.DataFrame(data=absent_interactions_feedback, columns=["userID", "itemID", "rating"])

negative_feedback_df.head()

,userID,itemID,rating
0,1034162,392595,0
1,203418,392595,0
2,1023669,392595,0
3,1290714,392595,0
4,168692,392595,0


In [13]:
## Merge the positive and negative feedback into one single master dataframe
prepared_dataset = pd.merge(negative_feedback_df, df, on=['userID', 'itemID'], how='outer').fillna(0).drop('rating_x', axis = 1)

# Cleaning up the column names
prepared_dataset.rename(columns = {'rating_y': 'rating'}, inplace = True)

prepared_dataset.head()

,userID,itemID,rating
0,1034162,392595,1.0
1,203418,392595,0.0
2,1023669,392595,0.0
3,1290714,392595,0.0
4,168692,392595,0.0


In [14]:
## Check number of positive and negative feedback samples
prepared_dataset['rating'].value_counts()

0.0    22284404
1.0        4996
Name: rating, dtype: int64

### Run and Evaluate Product Ranking Model

- We use the Cornac module the train and evaluate a Bayesian Personalised Ranking model
- We set the value for top K as 5 and train our model for 50 epochs
- We set the LR to 0.01
- We utilize 80% of our dataset for training and 20% for testing

In [15]:
## Call our BPR model train and evaluation script on our prepared dataset
result = run_bpr_model(data=prepared_dataset, k=10, epochs=20, learning_rate=0.001, train_size=0.8)

rating_threshold = 1.0
exclude_unknowns = False
---
Training data:
Number of users = 4872
Number of items = 4575
Number of ratings = 17831520
Max rating = 1.0
Min rating = 0.0
Global mean = 0.0
---
Test data:
Number of users = 4872
Number of items = 4575
Number of ratings = 4457880
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 4872
Total items = 4575

[BPR] Training started!


  0%|          | 0/20 [00:00<?, ?it/s]

Optimization finished!

[BPR] Evaluation started!


Ranking:   0%|          | 0/4872 [00:00<?, ?it/s]

In [16]:
## Capture the model metric results on test data
print(result)

    |  HR@10 |    MAP |    MRR | NDCG@10 | Train (s) | Test (s)
--- + ------ + ------ + ------ + ------- + --------- + --------
BPR | 0.0020 | 0.0015 | 0.0015 |  0.0007 |  163.7118 |   4.3571



# **References**

1. Data Preparation for Colborative Filtering | Microsoft
https://github.com/microsoft/recommenders/blob/main/examples/01_prepare_data/data_transform.ipynb

2. Cornac Movie Recommendation using BPR | Microsoft
https://github.com/microsoft/recommenders/blob/main/examples/02_model_collaborative_filtering/cornac_bpr_deep_dive.ipynb

3. Bayesian Personalised Ranking (BPR) Evaluation Example | PreferredAI, Cornac
https://github.com/PreferredAI/cornac/blob/master/examples/bpr_netflix.py
https://cornac.preferred.ai/

4. BPR: Bayesian personalized ranking from implicit feedback | Rendle, S., Freudenthaler, C., Gantner, Z., & Schmidt-Thieme, L. (2009, June).
https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf